In [1]:
# import needed modules
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib



# get adjusted closing prices of 5 selected companies with Quandl
quandl.ApiConfig.api_key = 'fbpNyXs-wE73L6r7s1Nc'
selected = ['CNP', 'F', 'WMT', 'GE', 'TSLA']
data = quandl.get_table('WIKI/PRICES', ticker = selected,
                        qopts = { 'columns': ['date', 'ticker', 'adj_close'] },
                        date = { 'gte': '2014-1-1', 'lte': '2016-12-31' }, paginate=True)
#data.head()

print(data)

# reorganise data pulled by setting date as index with
# columns of tickers and their corresponding adjusted prices
clean = data.set_index('date')
table = clean.pivot(index=clean.index, columns='ticker')
#table.head()

# calculate daily and annual returns of the stocks
returns_daily = table.pct_change() # Percentage change between the current and a prior element.
returns_annual = returns_daily.mean() * 250 #250 trading days

# get daily and covariance of returns of the stock
cov_daily = returns_daily.cov()
cov_annual = cov_daily * 250



# empty lists to store returns, volatility and weights of imaginary portfolios
port_returns = []
port_volatility = []
sharpe_ratio = []
stock_weights = []

# set the number of combinations for imaginary portfolios
num_assets = len(selected)
num_portfolios = 50000


#set random seed for reproduction's sake
np.random.seed(101)


# populate the empty lists with each portfolios returns,risk and weights
for single_portfolio in range(num_portfolios):
    weights = np.random.random(num_assets)
    weights /= np.sum(weights)
    returns = np.dot(weights, returns_annual)
    volatility = np.sqrt(np.dot(weights.T, np.dot(cov_annual, weights)))
    rf_rate = 0.00 # risk free rate of return
    sharpe = (returns - rf_rate)/ volatility
    sharpe_ratio.append(sharpe)
    port_returns.append(returns)
    port_volatility.append(volatility)
    stock_weights.append(weights)

# a dictionary for Returns and Risk values of each portfolio
portfolio = {'Returns': port_returns,
             'Volatility': port_volatility,
             'Sharpe Ratio': sharpe_ratio}



# extend original dictionary to accomodate each ticker and weight in the portfolio
for counter,symbol in enumerate(selected):
    portfolio[symbol+' Weight'] = [Weight[counter] for Weight in stock_weights]

# make a nice dataframe of the extended dictionary
df = pd.DataFrame(portfolio)


# get better labels for desired arrangement of columns
column_order = ['Returns', 'Volatility', 'Sharpe Ratio'] + [stock+' Weight' for stock in selected]

# reorder dataframe columns
df = df[column_order]

# find min Volatility & max sharpe values in the dataframe (df)
min_volatility = df['Volatility'].min()
max_sharpe = df['Sharpe Ratio'].max()

# use the min, max values to locate and create the two special portfolios
# df.loc --> Access a group of rows and columns by label(s) or a boolean array.
sharpe_portfolio = df.loc[df['Sharpe Ratio'] == max_sharpe] 
min_variance_port = df.loc[df['Volatility'] == min_volatility]


# plot frontier, max sharpe & min Volatility values with a scatterplot
plt.style.use('seaborn-dark')
fig, ax = plt.subplots()
df.plot.scatter(x='Volatility', y='Returns', c='Sharpe Ratio',
                cmap='RdYlGn', edgecolors='black', figsize=(10, 8), grid=True, ax=ax)
plt.scatter(x=sharpe_portfolio['Volatility'], y=sharpe_portfolio['Returns'], c='red', marker='D', s=200)
plt.scatter(x=min_variance_port['Volatility'], y=min_variance_port['Returns'], c='blue', marker='D', s=200 )
plt.xlabel('Volatility (Std. Deviation)')
plt.ylabel('Expected Returns')
plt.title('Optimal Portfolios')
plt.show()


# print the details of the 2 special portfolios
print(min_variance_port.T)
print(sharpe_portfolio.T)

# The most risk-averse investor would construct the minimum variance portfolio
# which has an expected return of 4.58% with an accompanying expected volatility of 13.86%. 

# Investors seeking the maximum risk-adjusted return would opt for portfolio that with 
# the maximum Sharpe Ratio which has an expected return of 11.61% with expected volatility pegged at 17.50%

# This method used to explain the concept of efficient frontier and optimal portfolios is the 
# Monte Carlo Simulation (sort of manual way) optimal portfolios.
    


           date ticker  adj_close
None                             
0    2014-01-02    CNP  19.116238
1    2014-01-03    CNP  19.107861
2    2014-01-06    CNP  19.132992
3    2014-01-07    CNP  19.334039
4    2014-01-08    CNP  19.132992
5    2014-01-09    CNP  19.350793
6    2014-01-10    CNP  19.585348
7    2014-01-13    CNP  19.300532
8    2014-01-14    CNP  19.459694
9    2014-01-15    CNP  19.652364
10   2014-01-16    CNP  19.736134
11   2014-01-17    CNP  19.618856
12   2014-01-21    CNP  19.861788
13   2014-01-22    CNP  19.928804
14   2014-01-23    CNP  19.744511
15   2014-01-24    CNP  19.392678
16   2014-01-27    CNP  19.417809
17   2014-01-28    CNP  19.551841
18   2014-01-29    CNP  19.392678
19   2014-01-30    CNP  19.727757
20   2014-01-31    CNP  19.602102
21   2014-02-03    CNP  19.308908
22   2014-02-04    CNP  19.317285
23   2014-02-05    CNP  19.300532
24   2014-02-06    CNP  19.417809
25   2014-02-07    CNP  19.543464
26   2014-02-10    CNP  19.602102
27   2014-02-1

                 17879
Returns       0.045828
Volatility    0.138552
Sharpe Ratio  0.330761
CNP Weight    0.240327
F Weight      0.104659
WMT Weight    0.257760
GE Weight     0.001487
TSLA Weight   0.395767
                 31209
Returns       0.116145
Volatility    0.175045
Sharpe Ratio  0.663514
CNP Weight    0.372890
F Weight      0.008482
WMT Weight    0.404987
GE Weight     0.211450
TSLA Weight   0.002190
